In [8]:
import os
import glob
import mne
import numpy as np
import pandas as pd
import joblib
from scipy import stats
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, cohen_kappa_score
import matplotlib.pyplot as plt

In [9]:
from scipy.stats import skew, kurtosis, entropy

def extract_emotion_features(data, sfreq):                                    # Sinyallere ait ozellikleri burada cikartiyoruz. data: np.array, shape (n_epochs, n_channels, n_times), sfreq: ornekleme hizi (Hz)

    feature_list = []   
    count=0                                                                 # Islenen toplam epoch sayisi.

    for epoch in data:                                                      # Her epoch icin dongu donuyoruz ve featurelarini cikartiyoruz.
        feats = []
        for ch_signal in epoch:                                             # Her kanal icin dongu

            mean = np.mean(ch_signal)                                       # Ortalama
            median = np.median(ch_signal)                                   # Medyan
            variance = np.var(ch_signal)                                    # Varyans
            min_val = np.min(ch_signal)                                     # Minimum deger
            max_val = np.max(ch_signal)                                     # Maksimum deger
            ptp = max_val - min_val                                         # Tepe araligi
            
            feats.extend([mean, median, variance, min_val, max_val, ptp])
            
            autocorr = np.correlate(ch_signal, ch_signal, mode='full')[len(ch_signal)-1:len(ch_signal)+5]       # Ilk 5 gecikme icin otokorelasyon degerleri
            feats.extend(autocorr[1:6])
            
            t = np.arange(len(ch_signal))                                   # Zaman vektoru
            slope = np.polyfit(t, ch_signal, 1)[0]                          # Egim hesabi
            feats.append(slope)
            
            hist, _ = np.histogram(ch_signal, bins=10, density=True)        # Entropi hesabi
            ent = entropy(hist)
            feats.append(ent)
            
            rms = np.sqrt(np.mean(ch_signal**2))                            # Root Mean Square frekans hesabi
            feats.append(rms)
            
            zcr = np.sum(np.diff(np.sign(ch_signal)) != 0) / len(ch_signal)     # Zero Crossing Rate hesabi
            feats.append(zcr)
            
            s = skew(ch_signal)                                            # Skewness ve kurtosis hesabi
            k = kurtosis(ch_signal)
            feats.extend([s, k])
            
            count+=1
        feature_list.append(feats)

    print (f"Özellik çıkarımı için toplam {count} epoch işleniyor...")
    return np.array(feature_list)

In [10]:
label_mapping = {
    'ibeg': 0, 'iend': 1, 'rsrt': 2, 'fixl': 3,
    'quiz': 4, 'qdon': 5, 'base': 6, 'bend': 7, 'trno': 8,
    'fixL': 9, 'stm': 10, 'clic': 11, 'vlnc': 12, 'arsl': 13,
    'dmns': 14, 'lkng': 15, 'fmrt': 16, 'relv': 17, 'cate': 18,
    'IBEG': 0, 'IEND': 1, 'puse': 19, 'boundary': 20, 'stop': 21,
    'baseline': 22,'baseend': 23,'trialno': 24,'fixation loop': 25,
    'neutral_1_1': 26,'valence': 27,'arousal': 13,'dominance': 14,'liking': 15,
    'familiarity': 16,'relevance': 17,'3_2': 28,'click': 29,
    'impedances begin': 30,'emotion_categ': 31,'impedances end': 32,'stop event': 33
}
def preprocess_emotion_data(eeg_file, event_file, epoch_duration=2.0):
    try:
        raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)                             # EEG verisini ve olaylari yukluyoruz.
        events_df = pd.read_csv(event_file, sep='\t')
        
        events = []
        for _, row in events_df.iterrows():                                                             # Etiketleri sayisallastiriyorum
            sample = int(row['onset'] * raw.info['sfreq']/1000)
            label_str = str(row['trial_type']).lower().strip()
            label = label_mapping.get(label_str, -1)
            if label >= 0:
                events.append([sample, 0, label])
        
        events_array = np.array(events, dtype=np.int64)
        
        epochs = mne.Epochs(                                                                            # Epochlari olusturuyorum.
            raw,
            events_array,
            tmin=0.0,
            tmax=epoch_duration,
            baseline=None,
            preload=True,
            reject_by_annotation=False
        )
        
        data = epochs.get_data()                                                                        # Epoch verilerinin ozelliklerini cikartiyorum.
        features = extract_emotion_features(data, raw.info['sfreq'])    
        labels = epochs.events[:, -1]
        
        return features, labels
        
    except Exception as e:
        print(f"Error processing {eeg_file}: {str(e)}")
        return np.array([]), np.array([])

In [11]:
def my_standard_scaler(X):              # Verilerimi 0-1 arasina standartize ediyorum.
    mean = np.mean(X, axis=0)           # Her sutunun ortalamasi
    std = np.std(X, axis=0)             # Her sutunun standart sapmasi
    X_scaled = (X - mean) / std         # Standardizasyon formulu
    return X_scaled

In [12]:
import joblib
def predict_emotions(eeg_file, event_file):
    features, true_labels = preprocess_emotion_data(eeg_file, event_file)                               # Veriyi onisleme uyguluyorum.
    if len(features) == 0:
        return {}
    
    features_scaled = my_standard_scaler(features)                                                      # Veriyi standartize ediyorum.
    
    rf_model = joblib.load('random_forest_model.joblib')                                                # Modelleri yukluyorum.
    xgb_model = joblib.load('xgboost_model.joblib')
    lstm_model = load_model('lstm_model.h5')
    mlp_model = load_model('mlp_model.h5')
    
    rf_pred = rf_model.predict(features_scaled)                                                         # Tahmin yaptiriyorum.
    xgb_pred = xgb_model.predict(features_scaled)
    
    timesteps = 1                                                                                       # LSTM icin veriyi tekrar sekillendiriyorum.
    features_per_timestep = features_scaled.shape[1] // timesteps
    X_reshaped = features_scaled.reshape(-1, timesteps, features_per_timestep)
    lstm_pred = np.argmax(lstm_model.predict(X_reshaped), axis=1)
    
    mlp_pred = np.argmax(mlp_model.predict(features_scaled), axis=1)

    mapping_inverse = {v: k for k, v in label_mapping.items()}                                          # Simdi tersten map ediyoruz, hata aliyordum burayi da ChatGPT'den aldim.
    true_labels_str = [mapping_inverse.get(x, 'UNKNOWN') for x in true_labels]
    rf_pred_str = [mapping_inverse.get(x, 'UNKNOWN') for x in rf_pred]
    xgb_pred_str = [mapping_inverse.get(x, 'UNKNOWN') for x in xgb_pred]
    lstm_pred_str = [mapping_inverse.get(x, 'UNKNOWN') for x in lstm_pred]
    mlp_pred_str = [mapping_inverse.get(x, 'UNKNOWN') for x in mlp_pred]
                                                                      
    results = {                                                                                         # Labellari decode ediyoruz.
        'true': true_labels_str,
        'rf': rf_pred_str,
        'xgb': xgb_pred_str,
        'lstm': lstm_pred_str,
        'mlp': mlp_pred_str
    }
    
    return results

In [13]:
def calculate_metrics(y_true, y_pred):                                          # Performans verilerini elde ediyorum.
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    kappa = cohen_kappa_score(y_true, y_pred)
    return acc, f1, kappa

In [14]:
def test_emotion_models(data_dir):
    import joblib
    
    le = joblib.load('label_encoder.joblib')                                    # Kaydettigimiz labelleri yukluyorum
    
    eeg_files = glob.glob(os.path.join(data_dir, "*.set"))                      # Dosyadaki EEG dosya ikililerini aliyorum.
    all_true, all_rf, all_xgb, all_lstm, all_mlp = [], [], [], [], []           # Tum modeller icin toplu sonuclari tutuyorum.
    for eeg_file in eeg_files:
        event_file = eeg_file.replace('_eeg.set', '_events.tsv')
        if not os.path.exists(event_file):
            print(f"Event file not found: {event_file}")
            continue
            
        print(f"Processing: {os.path.basename(eeg_file)}")
        results = predict_emotions(eeg_file, event_file)
        
        if not results:
            continue
            
        all_true.extend(results['true'])
        all_rf.extend(results['rf'])
        all_xgb.extend(results['xgb'])
        all_lstm.extend(results['lstm'])
        all_mlp.extend(results['mlp'])

    if len(all_true) == 0:
        print("Data işlenemedi.")
        return
    
    classes = le.classes_

    print("LabelEncoder sınıfları:", classes)
    print("LabelEncoder sınıf sayısı:", len(classes))
    print("Model tahmin unique değerleri (rf):", np.unique(all_rf))
    print("Model tahmin unique değerleri (xgb):", np.unique(all_xgb))
    print("True label unique değerleri:", np.unique(all_true))
    all_unique_labels = sorted(list(set(all_true) | set(all_rf) | set(all_xgb) | set(all_lstm) | set(all_mlp)))                     # Tum essiz labellari aliyorum.
    

    print("Random Forest Performance:")                                                                                             # Modellerin performanslarini hesapliyoruz.
    print(classification_report(all_true, all_rf, labels=all_unique_labels, target_names=all_unique_labels, zero_division=0))
    print("Confusion Matrix:")
    print(confusion_matrix(all_true, all_rf, labels=all_unique_labels))
    
    print("XGBoost Performance:")
    print(classification_report(all_true, all_xgb, labels=all_unique_labels, target_names=all_unique_labels, zero_division=0))
    print("Confusion Matrix:")
    print(confusion_matrix(all_true, all_xgb, labels=all_unique_labels))
    
    print("LSTM Performance:")
    print(classification_report(all_true, all_lstm, labels=all_unique_labels, target_names=all_unique_labels, zero_division=0))
    print("Confusion Matrix:")
    print(confusion_matrix(all_true, all_lstm, labels=all_unique_labels))
    
    print("MLP Performance:")
    print(classification_report(all_true, all_mlp, labels=all_unique_labels, target_names=all_unique_labels, zero_division=0))
    print("Confusion Matrix:")
    print(confusion_matrix(all_true, all_mlp, labels=all_unique_labels))
    

    rf_metrics = calculate_metrics(all_true, all_rf)                                                                               # Kullandigimiz metrikleri hesapliyoruz.
    xgb_metrics = calculate_metrics(all_true, all_xgb)
    lstm_metrics = calculate_metrics(all_true, all_lstm)
    mlp_metrics = calculate_metrics(all_true, all_mlp)
    
    print("MODEL PERFORMANS KARŞILAŞTIRMASI:")
    print(f"RF:   Accuracy={rf_metrics[0]:.4f}, F1={rf_metrics[1]:.4f}, Kappa={rf_metrics[2]:.4f}")
    print(f"XGB:  Accuracy={xgb_metrics[0]:.4f}, F1={xgb_metrics[1]:.4f}, Kappa={xgb_metrics[2]:.4f}")
    print(f"LSTM: Accuracy={lstm_metrics[0]:.4f}, F1={lstm_metrics[1]:.4f}, Kappa={lstm_metrics[2]:.4f}")
    print(f"MLP:  Accuracy={mlp_metrics[0]:.4f}, F1={mlp_metrics[1]:.4f}, Kappa={mlp_metrics[2]:.4f}")

label_mapping = {
    'ibeg': 0, 'iend': 1, 'rsrt': 2, 'fixl': 3,
    'quiz': 4, 'qdon': 5, 'base': 6, 'bend': 7, 'trno': 8,
    'fixL': 9, 'stm': 10, 'clic': 11, 'vlnc': 12, 'arsl': 13,
    'dmns': 14, 'lkng': 15, 'fmrt': 16, 'relv': 17, 'cate': 18,
    'IBEG': 0, 'IEND': 1, 'puse': 19, 'boundary': 20, 'stop': 21,
    'baseline': 22, 'baseend': 23, 'trialno': 24, 'fixation loop': 25,
    'neutral_1_1': 26, 'valence': 27, 'arousal': 13, 'dominance': 14, 'liking': 15,
    'familiarity': 16, 'relevance': 17, '3_2': 28, 'click': 29,
    'impedances begin': 30, 'emotion_categ': 31, 'impedances end': 32, 'stop event': 33
}

if __name__ == "__main__":
    test_data_dir = "./Emotion_EDF_Testing_Data"
    test_emotion_models(test_data_dir)

Processing: sub-mit081_task-Emotion_eeg.set


C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_7212\700143633.py:14: RuntimeWarning: Unknown types found, setting as type EEG:
pns: ['ECG', 'EMG', 'EMG_2']
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)                             # EEG verisini ve olaylari yukluyoruz.


Not setting metadata
135 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 135 events and 501 original time points ...
0 bad epochs dropped


C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_7212\700143633.py:14: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)                             # EEG verisini ve olaylari yukluyoruz.


Özellik çıkarımı için toplam 17820 epoch işleniyor...


...

C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_7212\700143633.py:14: RuntimeWarning: Unknown types found, setting as type EEG:
pns: ['ECG', 'EMG', 'EMG_2']
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)                             # EEG verisini ve olaylari yukluyoruz.


Not setting metadata
165 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 165 events and 501 original time points ...
0 bad epochs dropped


C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_7212\700143633.py:14: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)                             # EEG verisini ve olaylari yukluyoruz.
C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_7212\700143633.py:14: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(eeg_file, preload=True, verbose=False)                             # EEG verisini ve olaylari yukluyoruz.
C:\Users\EgeTERZI\AppData\Local\Temp\ipykernel_7212\924106973.py:4: RuntimeWarning: invalid value encountered in divide
  X_scaled = (X - mean) / std         # Standardizasyon formulu


Özellik çıkarımı için toplam 21780 epoch işleniyor...


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
LabelEncoder sınıfları: [ 0  1  3  8 10 11 12 13 14 15 16 17 18]
LabelEncoder sınıf sayısı: 13
Model tahmin unique değerleri (rf): ['IBEG' 'IEND' 'base' 'bend' 'clic' 'fixL' 'fixl' 'qdon' 'quiz' 'rsrt'
 'stm' 'trno' 'vlnc']
Model tahmin unique değerleri (xgb): ['IBEG' 'IEND' 'base' 'bend' 'clic' 'fixL' 'fixl' 'qdon' 'quiz' 'rsrt'
 'stm' 'trno' 'vlnc']
True label unique değerleri: ['IBEG' 'IEND' 'arousal' 'base' 'bend' 'boundary' 'cate' 'clic'
 'dominance' 'familiarity' 'fixl' 'liking' 'puse' 'qdon' 'quiz'
 'relevance' 'rsrt' 'stm' 'stop' 'trno' 'vlnc']
Random Forest Performance:
              precision    recall  f1-score   support

        IBEG       0.00      0.00      0.00         4
        IEND       0.00      0.00      0.00         4
     arousal       0.00      0.00      0.00        22
        base       0.00      0.00      0.00         2
        bend       0.00      0.00      0.00         2
    boundary 